In [1]:
import mlflow

In [2]:
from mlflow.tracking import MlflowClient                     # login to mLflow using MLflowclient class.

MLFLOW_TRACKING_URI = 'sqlite:///mlflow.db'                   # grant access to the sqlite artifact models and metadata.

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [4]:
# to check experiment with the client class. It outputs 2 experiments as seen in MLflow ui dashboard

client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/2. experiment-tracking/mlruns/3', creation_time=1710821129507, experiment_id='3', last_update_time=1710821129507, lifecycle_stage='active', name='first-nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1710730390938, experiment_id='0', last_update_time=1710730390938, lifecycle_stage='active', name='Default', tags={}>]

In [6]:
# create a new experiment in the MLflow ui dashboard using python api

client.create_experiment(name='my-cool-experiment')

'5'

In [7]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/2. experiment-tracking/mlruns/5', creation_time=1711160238654, experiment_id='5', last_update_time=1711160238654, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/2. experiment-tracking/mlruns/4', creation_time=1711160122502, experiment_id='4', last_update_time=1711160122502, lifecycle_stage='active', name='cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/2. experiment-tracking/mlruns/3', creation_time=1710821129507, experiment_id='3', last_update_time=1710821129507, lifecycle_stage='active', name='first-nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1710730390938, experiment_id='0', last_update_time=1710730390938, lifecycle_stage='active', name='Default', tags={}>]

In [38]:
# get the best active runs, ordered by rmse and without filter.

from mlflow.entities import ViewType

best_runs = client.search_runs(
    experiment_ids='3',
    filter_string='',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=4,
    order_by=['metrics.rmse ASC']
)

In [39]:
best_runs

[<Run: data=<RunData: metrics={'rmse': 6.7423033284974245}, params={'train-data-path': 'data/green_tripdata_2021-01.parquet',
  'val-data-path': 'data/green_tripdata_2021-02.parquet'}, tags={'developer': 'Eromosele',
  'mlflow.log-model.history': '[{"run_id": "4c0b7f27605e45ca8b14f7502cf73e52", '
                              '"artifact_path": "models", "utc_time_created": '
                              '"2024-03-22 22:25:50.931951", "flavors": '
                              '{"python_function": {"model_path": "model.pkl", '
                              '"predict_fn": "predict", "loader_module": '
                              '"mlflow.sklearn", "python_version": "3.10.13", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "sklearn": '
                              '{"pickled_model": "model.pkl", '
                              '"sklearn_version": "1.4.0", '
                              '"serializatio

In [43]:
# best run results above viewed nicely. it gives 4 output bcos max results was set to 4

for run in best_runs:
    print(f'run-id: {run.info.run_id}')

run-id: 4c0b7f27605e45ca8b14f7502cf73e52
run-id: 005192b72ed64163891853177fe5a10c
run-id: 121383de0e8d4b9bbc5c9975b54554a8
run-id: cb4903f280b443de8e7b33eb3d684bca


In [48]:
# get the best active runs, ordered by rmse and filtered by tags

from mlflow.entities import ViewType

best_runs = client.search_runs(
    experiment_ids='3',
    filter_string='tags.developer' == 'Eromosele',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=4,
    order_by=['metrics.rmse ASC']
)

In [49]:
# best run results above viewed nicely

for run in best_runs:
    print(f'run-id: {run.info.run_id}')

run-id: 4c0b7f27605e45ca8b14f7502cf73e52
run-id: 005192b72ed64163891853177fe5a10c
run-id: 121383de0e8d4b9bbc5c9975b54554a8
run-id: cb4903f280b443de8e7b33eb3d684bca


#### Promote models to model registry. input a model that's not in the registry

In [50]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [52]:
# register a new model into the model registry nyc-taxi-regressor

run_id = '121383de0e8d4b9bbc5c9975b54554a8'

model_uri = f'runs:/{run_id}/models'

mlflow.register_model(model_uri=model_uri, name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1711165902240, current_stage='None', description=None, last_updated_timestamp=1711165902240, name='nyc-taxi-regressor', run_id='121383de0e8d4b9bbc5c9975b54554a8', run_link=None, source=('/workspaces/mlops-zoomcamp/2. '
 'experiment-tracking/mlruns/3/121383de0e8d4b9bbc5c9975b54554a8/artifacts/models'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [53]:
# model_uri for the added model above into the registry.

model_uri

'runs:/121383de0e8d4b9bbc5c9975b54554a8/models'

#### Transition a model from one stage to another

In [57]:
#### list the versions of models in the model registry nyc-taxi-regresor

model_name = 'nyc-taxi-regressor'
latest_version = client.get_latest_versions(name=model_name)


latest_version

/tmp/ipykernel_35381/2412138472.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.1/model-registry.html#migrating-from-stages
  latest_version = client.get_latest_versions(name=model_name)


[<ModelVersion: aliases=[], creation_timestamp=1711165902240, current_stage='None', description=None, last_updated_timestamp=1711165902240, name='nyc-taxi-regressor', run_id='121383de0e8d4b9bbc5c9975b54554a8', run_link=None, source=('/workspaces/mlops-zoomcamp/2. '
  'experiment-tracking/mlruns/3/121383de0e8d4b9bbc5c9975b54554a8/artifacts/models'), status='READY', status_message=None, tags={}, user_id=None, version=3>]

In [60]:
# transition a model version 3 from none to staging

client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage='Staging',
    archive_existing_versions=False
)

/tmp/ipykernel_35381/3482683217.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1711165902240, current_stage='Staging', description=None, last_updated_timestamp=1711167459288, name='nyc-taxi-regressor', run_id='121383de0e8d4b9bbc5c9975b54554a8', run_link=None, source=('/workspaces/mlops-zoomcamp/2. '
 'experiment-tracking/mlruns/3/121383de0e8d4b9bbc5c9975b54554a8/artifacts/models'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [61]:
# transition a model version 1 from none to production

client.transition_model_version_stage(
    name=model_name, 
    version=1, 
    stage='Production', 
    archive_existing_versions=False)

/tmp/ipykernel_35381/3056626365.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(name=model_name, version=1, stage='Production', archive_existing_versions=False)


<ModelVersion: aliases=[], creation_timestamp=1711153413271, current_stage='Production', description='', last_updated_timestamp=1711168007264, name='nyc-taxi-regressor', run_id='4c0b7f27605e45ca8b14f7502cf73e52', run_link='', source=('/workspaces/mlops-zoomcamp/2. '
 'experiment-tracking/mlruns/3/4c0b7f27605e45ca8b14f7502cf73e52/artifacts/models'), status='READY', status_message=None, tags={'model': 'Gradient Boosting Regressor'}, user_id=None, version=1>

In [65]:
# transition a model version 2 from none to archived


client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage='Archived',
    archive_existing_versions=False
)

/tmp/ipykernel_35381/3451263181.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1711153726749, current_stage='Archived', description='', last_updated_timestamp=1711169034047, name='nyc-taxi-regressor', run_id='005192b72ed64163891853177fe5a10c', run_link='', source=('/workspaces/mlops-zoomcamp/2. '
 'experiment-tracking/mlruns/3/005192b72ed64163891853177fe5a10c/artifacts/models'), status='READY', status_message=None, tags={'model': 'Random Forest Regressor '}, user_id=None, version=2>

In [62]:
# to update description of model version

client.update_model_version(
    name=model_name,
    version=1,
    description='this model version is in Production stage'
)

<ModelVersion: aliases=[], creation_timestamp=1711153413271, current_stage='Production', description='this model version is in Production stage', last_updated_timestamp=1711168529875, name='nyc-taxi-regressor', run_id='4c0b7f27605e45ca8b14f7502cf73e52', run_link='', source=('/workspaces/mlops-zoomcamp/2. '
 'experiment-tracking/mlruns/3/4c0b7f27605e45ca8b14f7502cf73e52/artifacts/models'), status='READY', status_message=None, tags={'model': 'Gradient Boosting Regressor'}, user_id=None, version=1>